# 初始化

In [6]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold

"""Configuration"""
FEATURE_PATH = r'D:\jupyter-home\阿里恶意api检测\Data_small\feature\\'
RESULT_PATH = r'D:\jupyter-home\阿里恶意api检测\Data_small\result\\'


# XGBoost模型

In [ ]:
# 定义迭代次数，这个变量通常用于控制模型训练的轮数，
ROUND = 1000

# 配置XGBoost模型的参数，使用字典形式来设置各项参数，这些参数会影响XGBoost模型的训练过程和最终表现。
xgb_params = {
    # 指定XGBoost模型的目标函数，'multi:softprob'表示用于多分类问题，它会输出每个类别对应的概率值，方便后续进行分类决策以及评估模型性能等操作。
    'objective':'multi:softprob',
    # 明确分类的类别数量，这里设置为8，表示当前处理的是8分类问题。
    'num_class': 8,
    # eta参数也称为学习率，它控制着模型在每次迭代（如梯度下降等优化过程）中更新参数的步长大小。
    'eta': 0.04,
    # max_depth参数设定了决策树的最大深度，它决定了模型的复杂度和拟合能力。深度为6表示每棵决策树最多可以有6层节点。
    'max_depth': 6,
    # subsample参数用于指定在每次构建决策树时，随机抽取的样本比例。这里设置为0.9，表示每次构建树时会随机选取90%的训练样本，通过这种随机抽样的方式可以减少过拟合现象，增加模型的泛化能力，同时也能加快训练速度。
    'subsample': 0.9,
    # colsample_bytree参数控制着在构建每棵决策树时，随机抽取的特征比例。取值为0.7意味着每次构建树时会随机选择70%的特征，同样有助于防止过拟合，并且可以让模型关注到不同特征组合，提升模型的鲁棒性和多样性。
    'colsample_bytree': 0.7,
    # lambda参数是L2正则化项的系数，用于防止模型过拟合，通过对模型的权重参数添加平方惩罚项，使得模型的权重不会过大，让模型更加平滑，这里设置为2表示相对较强的L2正则化强度。
    'lambda': 2,
    # alpha参数是L1正则化项的系数，与L2正则化类似，也是为了防止过拟合，不过它对权重参数添加的是绝对值惩罚项，有助于产生稀疏的权重矩阵，选择特征等，这里设置为2同样体现了一定的正则化力度。
    'alpha': 2,
    # gamma参数用于控制是否进一步分裂节点，它是基于分裂带来的损失减少量来判断的。只有当分裂后损失减少量大于gamma设定的值（这里是1）时，才会进行节点分裂，这有助于避免不必要的树的生长，减少过拟合风险，同时简化模型结构。
    'gamma': 1,
    # scale_pos_weight参数用于处理类别不平衡问题，在某些情况下（比如正类样本数量远少于负类样本数量），可以通过设置这个参数来调整模型对不同类别样本的关注程度。这里设置为20，表示正类样本相对负类样本的权重比例，让模型在训练时更侧重正类样本的学习，以应对可能的类别不平衡情况。
    'scale_pos_weight': 20,
    # eval_metric参数指定了在模型训练过程中用于评估模型性能的指标，'mlogloss'表示多分类的对数损失，它衡量了模型预测概率分布与真实标签分布之间的差异，值越小说明模型预测越准确，可用于监控模型训练过程中的性能变化，以便选择合适的停止训练时机等。
    'eval_metric': 'mlogloss',
    # silent参数用于控制是否在训练过程中输出运行信息，设置为0表示输出相关信息（如训练进度、评估指标值等），方便查看模型训练的实时情况；若设置为1则表示静默模式，训练过程不会输出这些信息。
    'silent': 0,
    # seed参数即随机种子，设置为149用于确保每次运行模型时，只要代码和数据不变，生成的随机数序列是相同的，这样可以保证模型训练结果的可重复性，便于对比不同参数设置等情况下的模型性能。
    'seed': 149
}

## train函数

In [ ]:
def xgb_train(X_train, X_val, y_train, y_val, test, num_round):
    # 将训练集的特征数据和目标标签数据转换为XGBoost的DMatrix数据结构
    # DMatrix是XGBoost内部高效的数据存储和处理格式，能够加快模型训练和预测的速度，同时支持一些特定的数据处理功能
    # 这里传入X_train（特征数据）和y_train（标签数据）来构建用于训练的DMatrix对象dtrain
    dtrain = xgb.DMatrix(X_train, y_train)
    # 同样地，将验证集的特征数据和目标标签数据转换为DMatrix格式，得到用于验证的对象dval
    # 后续在训练过程中，可以通过这个dval在验证集上评估模型的性能表现，比如查看损失值、准确率等指标的变化情况
    dval = xgb.DMatrix(X_val, y_val)
    # 对于测试集数据test，先去掉其中可能存在的'file_id'列（因为'file_id'通常是样本的标识信息，不是模型用于预测的特征）
    # 然后将剩余的特征数据转换为DMatrix格式，得到用于最终测试预测的对象dtest
    dtest = xgb.DMatrix(test.drop(['file_id'], axis=1))

    # 创建一个监控列表watchlist，用于在模型训练过程中同时监控训练集和验证集的性能指标变化情况
    # 列表中的每个元素是一个元组，元组的第一个元素是DMatrix数据对象（如dtrain或dval），第二个元素是对应的名称（'train'表示训练集，'val'表示验证集）
    # 这样在训练时，XGBoost会根据设置输出相应数据集上的评估指标信息，方便观察模型的训练进度和效果
    watchlist = [(dtrain, 'train'), (dval, 'val')]

    # 使用xgb.train方法来训练XGBoost模型
    # xgb_params是之前定义好的模型参数字典，包含了目标函数、学习率、树的深度等各种超参数设置，这些参数会决定模型的结构和训练方式
    # dtrain是前面构建好的训练集DMatrix对象，作为训练数据输入
    # num_round指定了训练的轮数，即模型会基于训练数据进行这么多次的迭代训练
    # evals参数传入了watchlist，使得模型在训练过程中会按照设定的轮数间隔（由verbose_eval等参数控制）在训练集和验证集上计算评估指标，并输出相关信息
    # early_stopping_rounds参数设置为100，表示如果在连续100轮训练中，验证集上的评估指标（由eval_metric参数指定的指标，如mlogloss）没有得到改善，就提前停止训练
    # verbose_eval参数设置为100，意味着每训练100轮，就会输出一次训练集和验证集上的评估指标信息，方便查看训练过程中的模型性能变化情况
    model = xgb.train(xgb_params, dtrain, num_round, evals=watchlist, early_stopping_rounds=100, verbose_eval=100)
    
    # 使用model.predict方法对验证集（dval）进行预测，传入iteration_range参数来指定预测使用的迭代次数范围
    # 这里设置为(0, model.best_iteration + 1)，表示使用从第0次迭代到最佳迭代次数（包含最佳迭代次数那次）对应的树来进行预测
    p_val = pd.DataFrame(model.predict(dval, iteration_range=(0, model.best_iteration + 1)), index=X_val.index)
    p_test = pd.DataFrame(model.predict(dtest, iteration_range=(0, model.best_iteration + 1)), index=test.index)

    return (model, p_val, p_test)


# run


In [ ]:
print("Loading feature...")

# load feature v1
train_1 = pd.read_csv(FEATURE_PATH + 'train_base_features_v1.csv')
test_1 = pd.read_csv(FEATURE_PATH + 'test_base_features_v1.csv')

# load feature v2
train_2 = pd.read_csv(FEATURE_PATH + 'train_base_features_v2.csv')
test_2 = pd.read_csv(FEATURE_PATH + 'test_base_features_v2.csv')

# 以下代码块用于找出训练集特征（版本2）和测试集特征（版本2）中共同的列名，以便后续只保留共同的特征进行合并操作
# 这样做可以保证在合并特征时，训练集和测试集的特征维度一致，避免因特征不一致导致的模型训练或预测问题
interaction_feat = train_2.columns[train_2.columns.isin(test_2.columns.values)].values
train_2 = train_2[interaction_feat]
test_2 = test_2[interaction_feat]

# merge all features
# 将训练集的两种版本特征（train_1和train_2）按照'file_id'列进行左连接合并，把不同版本提取的特征整合到一起
# 左连接意味着以train_1的行索引为基础，将train_2中对应'file_id'的行合并过来，如果train_2中某些'file_id'不存在，则对应位置填充为缺失值
train = train_1.merge(train_2, on=['file_id'], how='left')
test = test_1.merge(test_2, on=['file_id'], how='left')

Loading feature ...


## train data prepare


In [16]:
# 从合并后的训练集数据train中，去掉'file_id'（通常是样本的标识列，不是模型用于学习的特征）和'label'（目标标签列，作为模型预测的目标，不作为输入特征）这两列
# 得到只包含输入模型的特征数据的DataFrame，赋值给X，后续将用X作为模型的特征输入进行训练
X = train.drop(['file_id', 'label'], axis=1)
# 从合并后的训练集数据train中提取'label'列，得到只包含目标标签的Series，赋值给y，y将作为模型训练时的目标值，用于告诉模型每个样本对应的真实类别
y = train['label']
X

,file_cnt,tid_distinct_cnt,api_distinct_cnt,tid_api_cnt_max,tid_api_cnt_min,tid_api_cnt_mean,tid_api_distinct_cnt_max,tid_api_distinct_cnt_min,tid_api_distinct_cnt_mean,AssignProcessToJobObject_index_min,...,recv_cnt_rate,recvfrom_cnt_rate,select_cnt_rate,send_cnt_rate,sendto_cnt_rate,setsockopt_cnt_rate,shutdown_cnt_rate,socket_cnt_rate,system_cnt_rate,timeGetTime_cnt_rate
0,57312,811,42,1016,18,70.668311,22,1,7.988903,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4173,30,108,858,1,139.100000,60,1,15.200000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2036,34,72,716,2,59.882353,45,2,10.294118,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2393,4,28,965,230,598.250000,16,10,13.500000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2691,37,77,632,1,72.729730,57,1,8.702703,NaN,...,NaN,NaN,NaN,NaN,NaN,0.015979,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13609,13,3,8,8,2,4.333333,5,2,3.333333,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13610,2,1,2,2,2,2.000000,2,2,2.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13611,1,1,1,1,1,1.000000,1,1,1.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13612,4,1,2,4,4,4.000000,2,2,2.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## add one_vs_rest prob

In [17]:
extra_feat_val = pd.read_csv(FEATURE_PATH + 'tr_lr_oof_prob.csv')
extra_feat_test = pd.read_csv(FEATURE_PATH + 'te_lr_oof_prob.csv')
# 通过列表推导式生成一个列名列表，列名格式为'prob'加上数字索引（这里范围是0到1，可能后续可扩展用于更多类别情况）
# 这些列名应该对应着前面读取的额外特征文件中的概率特征列名
prob_list = ['prob' + str(i) for i in range(1)]
# 将之前的特征数据X和额外特征文件中的对应概率特征列（通过prob_list选取）按照列方向进行合并
# 这样就为训练集特征数据增加了新的概率特征信息，丰富了输入模型的特征维度，赋值给X_extra，后续将用它作为更全面的训练集特征进行模型训练
X_extra = pd.concat([X, extra_feat_val[prob_list]], axis=1)
test_extra = pd.concat([test, extra_feat_test[prob_list]], axis=1)
# 打印提示信息，表示特征加载完成，告知使用者数据准备阶段已结束，可以进入模型训练等后续流程了
print("Loading complete")

Loading complete


In [18]:
X_extra

,file_cnt,tid_distinct_cnt,api_distinct_cnt,tid_api_cnt_max,tid_api_cnt_min,tid_api_cnt_mean,tid_api_distinct_cnt_max,tid_api_distinct_cnt_min,tid_api_distinct_cnt_mean,AssignProcessToJobObject_index_min,...,recvfrom_cnt_rate,select_cnt_rate,send_cnt_rate,sendto_cnt_rate,setsockopt_cnt_rate,shutdown_cnt_rate,socket_cnt_rate,system_cnt_rate,timeGetTime_cnt_rate,prob0
0,57312,811,42,1016,18,70.668311,22,1,7.988903,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.278899
1,4173,30,108,858,1,139.100000,60,1,15.200000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.400966
2,2036,34,72,716,2,59.882353,45,2,10.294118,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.322022
3,2393,4,28,965,230,598.250000,16,10,13.500000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.420037
4,2691,37,77,632,1,72.729730,57,1,8.702703,NaN,...,NaN,NaN,NaN,NaN,0.015979,NaN,NaN,NaN,NaN,0.293111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13609,13,3,8,8,2,4.333333,5,2,3.333333,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.439686
13610,2,1,2,2,2,2.000000,2,2,2.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.266177
13611,1,1,1,1,1,1.000000,1,1,1.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.275283
13612,4,1,2,4,4,4.000000,2,2,2.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.318935



## multi-class model training


In [19]:
# 创建一个空列表logloss_rlt，用于存储每次交叉验证折叠（fold）中模型在验证集上的对数损失（log_loss）值，方便后续分析模型性能的稳定性等情况
logloss_rlt = []
# 创建一个空的DataFrame p_val_all，用于在后续的交叉验证过程中，逐步合并每次折叠的验证集预测概率结果和真实标签信息
# 最终可以通过这个DataFrame全面查看模型在各个折叠验证集上的表现情况
p_val_all = pd.DataFrame()
# 创建一个初始值全为0的DataFrame，形状为（测试集样本数量，8），用于在后续的交叉验证过程中逐步累加每次折叠对测试集的预测概率结果
# 这里的8应该对应着多分类任务中的类别数量，意味着为每个类别都预留了位置来存储预测概率
p_test_all = pd.DataFrame(np.zeros((test.shape[0], 8)))
# 创建一个分层交叉验证对象skf，设置将数据划分为5折（n_splits=5），通过random_state参数固定随机种子为4，确保每次运行代码时数据划分结果一致
# shuffle=True表示在划分数据前先对数据进行随机打乱，使得每次划分的样本更具随机性和代表性
skf = StratifiedKFold(n_splits=5, random_state=4, shuffle=True)

## start 5-fold CV

In [20]:
# 遍历分层交叉验证的每一次折叠（fold），每次循环会得到本次折叠对应的训练集索引tr_index和验证集索引val_index
for fold_i, (tr_index, val_index) in enumerate(skf.split(X, y)):
    print('FOLD -', fold_i, ' Start...')
    
    # Prepare train, val dataset
    # 根据训练集索引tr_index和验证集索引val_index，从包含额外特征的训练集数据X_extra中选取对应的行，分别得到本次折叠的训练集特征数据X_train和验证集特征数据X_val
    X_train, X_val = X_extra.iloc[tr_index, :], X_extra.iloc[val_index, :]
    # 根据索引从目标标签数据y中选取对应的标签值，分别得到本次折叠的训练集标签y_train和验证集标签y_val
    y_train, y_val = y[tr_index], y[val_index]

    # Train model
    model, p_val, p_test = xgb_train(X_train, X_val, y_train, y_val, test_extra, ROUND)
    
    # Evaluate Model and Concatenate Val-Prediction
    # 使用log_loss函数计算模型在本次折叠验证集上的对数损失值
    # 该指标衡量了模型预测概率分布与真实标签分布之间的差异，值越小说明模型预测越准确
    m_log_loss = log_loss(y_val, p_val)
    print('----------------log_loss : ', m_log_loss, ' ---------------------')
    # 将本次折叠计算得到的对数损失值添加到logloss_rlt列表中，用于后续统计分析模型在不同折叠下的性能表现
    logloss_rlt = logloss_rlt + [m_log_loss]
    # 将本次折叠的验证集真实标签y_val和预测概率结果p_val按照列方向进行合并，形成一个包含真实标签和预测概率的DataFrame
    truth_prob_df = pd.concat([y_val, p_val], axis=1)
    # 将上述包含真实标签和预测概率的DataFrame合并到总的验证集结果DataFrame p_val_all中，方便后续整体查看和分析模型在所有折叠验证集上的表现
    p_val_all = pd.concat([p_val_all, truth_prob_df], axis=0)
    
    # Predict Test Dataset
    # 将本次折叠对测试集的预测概率结果p_test按一定权重（这里是0.2）累加到总的测试集预测结果DataFrame p_test_all中
    # 通过多次折叠的累加平均等操作，可以得到更稳定、更可靠的测试集最终预测结果
    p_test_all = p_test_all + 0.2 * p_test

FOLD - 0  Start...


C:\Users\raoxinyu\AppData\Roaming\Python\Python312\site-packages\xgboost\core.py:158: UserWarning: [11:34:28] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "scale_pos_weight", "silent" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	train-mlogloss:1.98852	val-mlogloss:1.99247
[100]	train-mlogloss:0.39866	val-mlogloss:0.54399
[200]	train-mlogloss:0.24761	val-mlogloss:0.45597
[300]	train-mlogloss:0.20353	val-mlogloss:0.43401
[400]	train-mlogloss:0.18920	val-mlogloss:0.42614
[500]	train-mlogloss:0.18588	val-mlogloss:0.42451
[600]	train-mlogloss:0.18424	val-mlogloss:0.42390
[700]	train-mlogloss:0.18254	val-mlogloss:0.42323
[800]	train-mlogloss:0.18168	val-mlogloss:0.42311
[900]	train-mlogloss:0.18073	val-mlogloss:0.42300
[999]	train-mlogloss:0.18022	val-mlogloss:0.42284
----------------log_loss :  0.4228433301618407  ---------------------
FOLD - 1  Start...


C:\Users\raoxinyu\AppData\Roaming\Python\Python312\site-packages\xgboost\core.py:158: UserWarning: [11:36:51] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "scale_pos_weight", "silent" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	train-mlogloss:1.98798	val-mlogloss:1.99395
[100]	train-mlogloss:0.39586	val-mlogloss:0.56426
[200]	train-mlogloss:0.24324	val-mlogloss:0.46669
[300]	train-mlogloss:0.20117	val-mlogloss:0.44487
[400]	train-mlogloss:0.18786	val-mlogloss:0.43831
[500]	train-mlogloss:0.18473	val-mlogloss:0.43678
[600]	train-mlogloss:0.18279	val-mlogloss:0.43630
[700]	train-mlogloss:0.18160	val-mlogloss:0.43601
[800]	train-mlogloss:0.18096	val-mlogloss:0.43562
[900]	train-mlogloss:0.18020	val-mlogloss:0.43545
[999]	train-mlogloss:0.17956	val-mlogloss:0.43514
----------------log_loss :  0.43513296146462355  ---------------------
FOLD - 2  Start...


...

[0]	train-mlogloss:1.98845	val-mlogloss:1.99129
[100]	train-mlogloss:0.39744	val-mlogloss:0.54263
[200]	train-mlogloss:0.24939	val-mlogloss:0.44997
[300]	train-mlogloss:0.20693	val-mlogloss:0.42823
[400]	train-mlogloss:0.19223	val-mlogloss:0.42080
[500]	train-mlogloss:0.18830	val-mlogloss:0.41917
[600]	train-mlogloss:0.18679	val-mlogloss:0.41854
[700]	train-mlogloss:0.18545	val-mlogloss:0.41813
[800]	train-mlogloss:0.18467	val-mlogloss:0.41782
[900]	train-mlogloss:0.18380	val-mlogloss:0.41732
[999]	train-mlogloss:0.18308	val-mlogloss:0.41707
----------------log_loss :  0.4170687033166101  ---------------------
FOLD - 3  Start...


C:\Users\raoxinyu\AppData\Roaming\Python\Python312\site-packages\xgboost\core.py:158: UserWarning: [11:41:39] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "scale_pos_weight", "silent" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	train-mlogloss:1.98890	val-mlogloss:1.99326
[100]	train-mlogloss:0.39972	val-mlogloss:0.52666
[200]	train-mlogloss:0.24826	val-mlogloss:0.43031
[300]	train-mlogloss:0.20452	val-mlogloss:0.40848
[400]	train-mlogloss:0.19097	val-mlogloss:0.40133
[500]	train-mlogloss:0.18721	val-mlogloss:0.39962
[600]	train-mlogloss:0.18483	val-mlogloss:0.39877
[700]	train-mlogloss:0.18341	val-mlogloss:0.39820
[800]	train-mlogloss:0.18269	val-mlogloss:0.39792
[900]	train-mlogloss:0.18199	val-mlogloss:0.39762
[999]	train-mlogloss:0.18159	val-mlogloss:0.39742
----------------log_loss :  0.3974177862938347  ---------------------
FOLD - 4  Start...


C:\Users\raoxinyu\AppData\Roaming\Python\Python312\site-packages\xgboost\core.py:158: UserWarning: [11:44:09] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "scale_pos_weight", "silent" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	train-mlogloss:1.98900	val-mlogloss:1.99172
[100]	train-mlogloss:0.40267	val-mlogloss:0.53438
[200]	train-mlogloss:0.25303	val-mlogloss:0.44342
[300]	train-mlogloss:0.20852	val-mlogloss:0.41914
[400]	train-mlogloss:0.19405	val-mlogloss:0.41075
[500]	train-mlogloss:0.19015	val-mlogloss:0.40861
[600]	train-mlogloss:0.18802	val-mlogloss:0.40781
[700]	train-mlogloss:0.18663	val-mlogloss:0.40724
[800]	train-mlogloss:0.18533	val-mlogloss:0.40673
[900]	train-mlogloss:0.18436	val-mlogloss:0.40626
[999]	train-mlogloss:0.18376	val-mlogloss:0.40606
----------------log_loss :  0.4060577750348953  ---------------------


## generate submit file


In [ ]:
# 将测试集的'file_id'列和最终整合得到的测试集预测概率结果p_test_all按照列方向进行合并，形成一个包含样本标识和各类别预测概率的DataFrame
rlt = pd.concat([test['file_id'], p_test_all], axis=1)
# 生成与预测概率列对应的列名列表，格式为'prob'加上数字索引（从0到7，对应8个类别），用于给最终的结果DataFrame设置合适的列名
prob_list = ['prob' + str(i) for i in range(8)]
# 给合并后的结果DataFrame rlt设置列名，第一列是'file_id'，后面依次是各概率列名，使其列名更具表意性，符合后续使用或提交结果的要求
rlt.columns = ['file_id'] + prob_list
rlt.to_csv(RESULT_PATH + '/submit.csv', index=None)

In [23]:
# 将概率最高的类别作为预测标签
predicted_labels = p_test_all.idxmax(axis=1)

# 将预测标签添加到结果DataFrame中
rlt_2 = pd.concat([test['file_id'], predicted_labels], axis=1)
rlt_2.columns = ['file_id', 'predicted_label']
rlt_2.to_csv(RESULT_PATH + '/predict_label.csv', index=None)